# 8_特徴量エンジニアリング
- 前章では「性別」「年齢」「運賃」という3つのデータから生死の予測をたてました
- しかし、「性別」のみで予測した場合との精度の差はみられなかったと思います
- ここで取り組む特徴量エンジニアリングとは、予測モデルに使う特徴量自体を自ら作り出し、精度向上を狙うものです
- 今回は、Cabin（部屋番号）を用いた特徴量エンジニアリングを行います。  
- 「予測モデル構築に使うデータ：train.csv」、「モデルの検証に使うデータ：test.csv」

## ■写経

### ライブラリインポート（PandasとSciPy）
- 表形式データを簡単に扱うためのPandasをプログラムにインポートします。インポートすることでPandasの機能が有効化されます
- また予測モデルの構築のためにsklearnをインポートします

In [72]:
import pandas as pd
from sklearn.linear_model import LinearRegression
from sklearn.impute import SimpleImputer

### データ読み込み（前章と同じ）
- Pandasの機能を使い、"train.csv"というデータをPythonコードで読み込み、Dataframeオブジェクトとして扱います

In [73]:
df_train = pd.read_csv("train.csv")

### データを見てみる（前章と同じ）
- head関数をつかって、先頭5行分のデータをダイジェスト表示します

In [74]:
df_train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


### 前準備
- maleを0、femaleを1に置き換えた列を作成します
- Cabin列の頭文字が部屋グレードを表すため、各行のCabinの頭文字を取得し、ダミー変数化します
  - ダミー変数化とは、質的データの各水準をデータカラムとし、2値変数とする操作です。これにより質的データを回帰モデルのような連続値関数で取り扱えるようになります

In [75]:
# Sexを数値に変換(男性:0, 女性:1)
df_train["Sex_bi"] = df_train["Sex"].map({"male":0, "female":1})

# Cabinの頭文字を取得
# pd.notnull(x)はxが欠損値でない場合にTrueを返す
# lumbda x: x[0]はxの1文字目を取得
df_train["Cabin_initial"] = df_train["Cabin"].apply(lambda x: x[0] if pd.notnull(x) else "N")

# get_dummiesでone-hotエンコーディング
#df_train = pd.get_dummies(df_train,columns=["Cabin_initial"])


In [76]:
df_train = pd.get_dummies(df_train, columns=["Cabin_initial"])

In [79]:
df_train

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,...,Sex_bi,Cabin_initial_A,Cabin_initial_B,Cabin_initial_C,Cabin_initial_D,Cabin_initial_E,Cabin_initial_F,Cabin_initial_G,Cabin_initial_N,Cabin_initial_T
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,...,0,False,False,False,False,False,False,False,True,False
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,...,1,False,False,True,False,False,False,False,False,False
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,...,1,False,False,False,False,False,False,False,True,False
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,...,1,False,False,True,False,False,False,False,False,False
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,...,0,False,False,False,False,False,False,False,True,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,...,0,False,False,False,False,False,False,False,True,False
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,...,1,False,True,False,False,False,False,False,False,False
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,...,1,False,False,False,False,False,False,False,True,False
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,...,0,False,False,True,False,False,False,False,False,False


In [ ]:
df_train["Cabin"]

### 重回帰モデルで使用する特徴量と目的変数を定義し、欠損値処理をする
- 今回は性別、年齢、客室ランク（特徴量）が生死（目的変数）と関係するという仮説をたてています

In [83]:
# 欠損値保管は最頻値とする
imputer = SimpleImputer(strategy="most_frequent")

# 特徴量の選択。Sex_bi列とCabinのダミー変数
features = ["Sex_bi"]+[col for col in df_train.columns if "Cabin_initial" in col]
X_train = df_train[features]
y_train = df_train["Survived"]

# Fill missing values
X_train = imputer.fit_transform(X_train)

In [85]:
features

['Sex_bi',
 'Cabin_initial_A',
 'Cabin_initial_B',
 'Cabin_initial_C',
 'Cabin_initial_D',
 'Cabin_initial_E',
 'Cabin_initial_F',
 'Cabin_initial_G',
 'Cabin_initial_N',
 'Cabin_initial_T']

### 重回帰モデルの作成と学習

In [86]:
# 空の予測モデルを作成
model = LinearRegression()

# 学習データを与えて学習
model.fit(X_train, y_train)

LinearRegression()

### テストデータに対する予測の実施
- 上記で学習したモデルに、テスト用のデータを予測させます

In [88]:
# 提出用テストデータを読み込みます（すなわち、学習したモデルの精度を測るための検証データセットです）
df_test = pd.read_csv("test.csv")

# 提出用データも同様に、'Sex'列を0、1に変換します
df_test["Sex_bi"] = df_test["Sex"].map({"male":0, "female":1})

df_test["Cabin_initial"]=df_test["Cabin"].apply(lambda x: x[0] if pd.notnull(x) else "N")
df_test = pd.get_dummies(df_test, columns=["Cabin_initial"])

# 提出用データの特徴量を分割します
for col in features:
    if col not in df_test.columns:
        df_test[col] = 0
X_test = df_test[features]
X_test = imputer.transform(X_test)

# 提出用データに対する予測を行います
predictions = model.predict(X_test)

#0-1 0.5以上なら1にする


# predictionsの各データを四捨五入してpredictionsに再代入する
predictions = [round(pred) for pred in predictions]


In [89]:
predictions

[0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,


### 提出用データの作成
- 提出データは、passenger id, Survivedという2列のデータ形式とします
- データをcsv形式で出力します

In [90]:
# 提出用のDataFrameを作成します
submission = pd.DataFrame({"PassengerId": df_test["PassengerId"], "Survived": predictions})

# 提出用のcsvファイルを作成します
submission.to_csv("submission_8.csv", index=False)


: 

### 答え合わせ
- この問題は、世界的なAIコンペサイトであるKaggleの練習問題コンペになっています
- そこでKaggleに上記の結果を提出して、スコア（予測精度）を確認しましょう
- https://www.kaggle.com/competitions/titanic/overview